In [ ]:
pip install -e .

In [ ]:
import pandas as pd
import numpy as np
import PharmOps
from importlib.metadata import version

In [ ]:
# import raw data spreadsheet
rawData = pd.read_excel('sample data\\Binding Template for RAW transformations.xlsx')
rawData

In [ ]:
# 8 rows, labeled A-H in unnamed first column. 12 columns of wells
rowLabel = rawData.loc[rawData.index[0:8], ['Unnamed: 0']]
rawValues = rawData.loc[rawData.index[0:8], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]
print(rawValues)
print(rowLabel)

In [ ]:
# data loading
testBind = PharmOps.WellData(rawData)
testBind.display()

In [ ]:
# object serialization and saving
import pickle
import h5py

filePath = "sample data/testH5.h5"
serializedObj = pickle.dumps(testBind)
serializedArray = np.frombuffer(serializedObj, dtype='uint8')
groupName = "testObjects"
datasetName = "testData"
with h5py.File(filePath, "a") as h5file:
    group = h5file.require_group(groupName)
    if datasetName in group:
        del group[datasetName]
        print(f"Dataset '{datasetName}' exists. Overwriting.")
    group.create_dataset("testData", data=np.frombuffer(serializedObj, dtype='uint8'))
    metadata = h5file.require_group("metadata")
    currentVersion = version("PharmOps")
    metadata.attrs["version"] = currentVersion

In [ ]:
# object loading and deserialization
with h5py.File(filePath, "r") as h5file:
    if groupName not in h5file or datasetName not in h5file[groupName]:
        raise KeyError(f"Dataset '{datasetName}' not found in group '{groupName}'")
    
    serializedArray = h5file[groupName][datasetName][:]
    loadedRawData = pickle.loads(serializedArray.tobytes())
    print(list(h5file.keys()))

loadedRawData.display()

In [ ]:
# GUI sandbox
import tkinter as tk
from tkinter import ttk

root = tk.Tk()
root.title("binding GUI test")
root.geometry("625x300")

mainframe = ttk.Frame(root)
mainframe['borderwidth'] = 10
mainframe['relief'] = 'sunken'
mainframe.grid(row=0, column=0, sticky="N, S, E, W", padx=5, pady=5)
root.grid_rowconfigure(0, weight=1)
root.grid_columnconfigure(0, weight=1)


currentObj = loadedRawData
tree = ttk.Treeview(mainframe, columns=list(currentObj.data.columns), show="headings")
tree.grid(row=0, column=0, sticky="N, S, E, W")

for col in currentObj.data.columns:
    tree.heading(col, text=col)
    tree.column(col, width=100, anchor=tk.CENTER)

for _, row in currentObj.data.iterrows():
    tree.insert("", tk.END, values=list(row))

mainframe.grid_rowconfigure(0, weight=1)
mainframe.grid_columnconfigure(0, weight=1)

scroll_x = ttk.Scrollbar(mainframe, orient=tk.HORIZONTAL, command=tree.xview)
scroll_x.grid(row=1, column=0, sticky="E, W")
scroll_y = ttk.Scrollbar(mainframe, orient=tk.VERTICAL, command=tree.yview)
scroll_y.grid(row=0, column=1, sticky="N, S")

tree.configure(xscrollcommand=scroll_x.set, yscrollcommand=scroll_y.set)

label = ttk.Label(root, text="more text")
label.grid(row=1, column=0, sticky="W", padx=5, pady=5)

button = ttk.Button(root, text="Click me")
button.grid(row=2, column=0, sticky="W", padx=5, pady=5)
root.mainloop()